# RAG Pipeline

1. gpt4all embeddings
2. llama2; 7b params; 6 bits quanitization

# VectorDB Setup

In [1]:
# !pip3 install --upgrade --quiet  langchain langchain-community langchainhub gpt4all chromadb 
# !pip3 install llama-cpp-python

/bin/bash: pip3: command not found


In [1]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# data = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)

/Users/rohitmalhotra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)


# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function = SentenceTransformerEmbeddings(model_name="Salesforce/SFR-Embedding-Mistral")


# vectorstore = Chroma(persist_directory="gpt4allDB", embedding_function=GPT4AllEmbeddings())
# vectorstore = Chroma(persist_directory="all-MiniLM-L6-v2DB", embedding_function=embedding_function)
vectorstore = Chroma(persist_directory="sfr-mistralDB/", embedding_function=embedding_function)



bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [1]:
# VectorDB Search Test

question = "What is buggy?"
docs = vectorstore.similarity_search(question)
len(docs)

NameError: name 'vectorstore' is not defined

# Chat Agent Setup

In [ ]:
!pip install --upgrade --quiet  llama-cpp-python
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q6_K.gguf

In [ ]:
from langchain_community.llms import LlamaCpp


n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q6_K.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
)


# RAG

In [ ]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnablePick
from langchain_core.prompts.chat import HumanMessagePromptTemplate, PromptTemplate

rag_prompt = hub.pull("rlm/rag-prompt")
# rag_prompt.messages
prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use one sentence maximum and keep the answer CONCISE. Keep the answer CONCISE.\nQuestion: {question} \nContext: {context} \nAnswer:"))
rag_prompt.messages = [prompt]

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


retriever = vectorstore.as_retriever()
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)



In [ ]:
f = open("SubmissionData/test/questions.txt", "r")
questions = f.readlines()
f.close()

questions = [i.strip() for i in questions]

In [ ]:
from tqdm import tqdm

answers = []
for i in tqdm(range(len(questions))):
  response = qa_chain.invoke(questions[i])
  answers.append(response)

In [ ]:
def writeListToFile(filename, l):
    f = open(filename, "w")
    for i in l:
        f.write(i + "\n")
    f.close()



writeListToFile("SubmissionData/system_outputs/system_output_3.txt", answers)